In [1]:
from datasets import load_from_disk, load_dataset

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# https://huggingface.co/HuggingFaceH4

In [ ]:
# https://github.com/yaodongC/awesome-instruction-dataset

In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

class Translator:
    def __init__(self, 
        model_name: str
    ):
        self.model_name = model_name
        self.init()
        
        self.model = None
        self.tokenizer = None
    
    def init(self):
        print("Init model.")
        if self.model_name == "facebook/nllb-200-3.3B":
            self.model = AutoModelForSeq2SeqLM.from_pretrained(
                self.model_name, 
                use_auth_token=True,
            )
            self.model.eval()
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.model_name,
                use_auth_token=True,
            )
        print("Model is initialized.")
    
    def translate(self, text: str):
        with torch.no_grad():
            if self.model_name == "facebook/nllb-200-3.3B":
                return self.nllb_translate(text=text)
    
    def nllb_translate(self, text: str):
        inputs = self.tokenizer(text, return_tensors="pt")
        inputs = self.to_cuda(inputs=inputs)
        translated_tokens = self.model.generate(
            **inputs, forced_bos_token_id=self.tokenizer.lang_code_to_id["rus_Cyrl"], max_length=30
        )
        return self.tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

    def to_cuda(self, inputs):
        for key in inputs.keys():
            inputs[key] = inputs[key].to("cuda")
            return inputs
         

In [8]:
model_name = 'facebook/nllb-200-3.3B'
translator = Translator(
	model_name=model_name
)

Loading checkpoint shards: 100%|██████████| 3/3 [00:31<00:00, 10.41s/it]


In [ ]:
"""
- databricks/databricks-dolly-15k - https://huggingface.co/datasets/databricks/databricks-dolly-15k
- self_instruct - https://huggingface.co/datasets/yizhongw/self_instruct
- OpenAssistant/oasst1 - https://huggingface.co/datasets/OpenAssistant/oasst1
- sahil2801/CodeAlpaca-20k - https://huggingface.co/datasets/sahil2801/CodeAlpaca-20k
- HuggingFaceH4/helpful_instructions - https://huggingface.co/datasets/HuggingFaceH4/helpful_instructions
- HuggingFaceH4/stack-exchange-preferences - https://huggingface.co/datasets/HuggingFaceH4/stack-exchange-preferences
- openai/summarize_from_feedback - https://huggingface.co/datasets/openai/summarize_from_feedback 
- stanfordnlp/SHP - https://huggingface.co/datasets/stanfordnlp/SHP
- unnatural-instructions(text-davinci002) - https://github.com/orhonovich/unnatural-instructions/tree/main
- eli5 - https://huggingface.co/datasets/eli5
- chip2_instruct_alpha - https://github.com/Rallio67/language-model-agents/tree/main

- нужно больше датасетов с саммаризацией
"""


### databricks/databricks-dolly-15k

In [3]:
data = load_dataset("databricks/databricks-dolly-15k")
data

Found cached dataset json (/home/kosenko/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-aae1918f8081f1c6/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)
100%|██████████| 1/1 [00:00<00:00, 700.92it/s]


DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 15014
    })
})

In [4]:
data['train'][0]

{'instruction': 'When did Virgin Australia start operating?',
 'context': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.[3] It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.[4]",
 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.',
 'category': 'closed_qa'}